In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print("V100 is three times faster than P100")
  print(gpu_info)

V100 is three times faster than P100
Sun Dec 13 17:00:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                   

In [4]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/Deep Learning Project/Interpretable-activity-prediction"
# !ls

Mounted at /content/drive
/content/drive/MyDrive/Deep Learning Project/Interpretable-activity-prediction


In [55]:
import torch
import numpy as np
import scipy.sparse as sp
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn import model_selection
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, confusion_matrix
from statistics import *



## Generate normalized adjacency matrix and normalize the feature matrix

In [6]:
def normalized_adjacency(adj):
    adj = adj + sp.eye(adj.shape[0])
    adj = sp.coo_matrix(adj)
    row_sum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(row_sum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()

def row_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(x.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return x


## Preprocess input features

In [7]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


def sgc_precompute(features, adj, degree):
    adj = normalized_adjacency(adj)
    adj = sparse_mx_to_torch_sparse_tensor(adj).float()
    features = torch.tensor(features)
    for i in range(degree):
        features = torch.spmm(adj, features)
    return features   # aton_num * 133



# Load data

In [14]:
class MyDataset(Dataset):
    def __init__(self, X, A, Y, degree):
        self.X = X
        self.A = A
        self.Y = Y
        self.degree = degree
    def __len__(self):
        return len(self.Y)
    def __getitem__(self, index):
        X = sgc_precompute(self.X[index], self.A[index], self.degree)
        return torch.sum(X, dim = 0, keepdim = False), torch.tensor(self.Y[index])


class MyDataset_mol(Dataset):   # molecular feature added
    def __init__(self, X, A, Y, degree, mol_feat):
        self.X = X
        self.A = A
        self.Y = Y
        self.degree = degree
        self.mol_feat = mol_feat

    def __len__(self):
        return len(self.Y)
    def __getitem__(self, index):
        X = sgc_precompute(self.X[index], self.A[index], self.degree)
        # sum_atom_feat = torch.sum(X, dim = 0)
        sum_atom_feat = torch.mean(X, dim = 0)
        all_feat = np.concatenate( (self.mol_feat[index] , sum_atom_feat), axis=0)
        # print("sum_atom_feat", sum_atom_feat)
        # print("self.mol_feat", np.array(self.mol_feat[index]))
        # print("all_feat", all_feat)
        # return torch.tensor(all_feat).float(), torch.tensor(self.Y[index])
        return sum_atom_feat, torch.tensor(self.Y[index])



# Essemble training

In [74]:
def train(models, train_dataloaders, val_dataloaders, optimizers, schedulers, criterion, num_epoch, k, ensemble):
    train_losses_list, val_losses_list, train_accs_list, val_accs_list, auc_scores_list = [], [], [], [], []
    for i in range(ensemble):
        print("training ensemble model", i+1)

        model = models[i]
        train_dataloader = train_dataloaders[i]
        val_dataloader = val_dataloaders[i]
        optimizer = optimizers[i]
        scheduler = schedulers[i]

        true_labels = np.array([])
        probs = np.array([])

        train_losses = []
        val_losses = []
        train_accs = []
        val_accs = []
        for epoch in range(num_epoch):
            model.train()
            correct = 0
            total = 0
            running_loss = 0
            for batch_idx, (X, Y) in enumerate(train_dataloader):
                optimizer.zero_grad()
                output = model.forward(X)
                probs = np.append(probs, output[:,-1].cpu().detach().numpy())
                true_labels = np.append(true_labels, Y.cpu().detach().numpy())
                
                pred = torch.argmax(output, dim = 1)

                correct += (pred==Y).sum().item()
                total += Y.size(0)
                train_acc = correct/total
                loss = criterion(output, Y) 
                running_loss += loss.item() * Y.size(0)
                train_loss = running_loss/total
                loss.backward()
                optimizer.step()
                
            train_accs.append(train_acc)
            train_losses.append(train_loss)
            val_acc, val_loss, _ = validate(model, val_dataloader, criterion, k)
            scheduler.step(val_loss)
            val_accs.append(val_acc)
            val_losses.append(val_loss)
            auc_scores = roc_auc_score(true_labels , probs) 

        train_losses_list.append(train_losses)
        val_losses_list.append(val_losses)
        train_accs_list.append(train_accs)
        val_accs_list.append(val_accs)
        auc_scores_list.append(auc_scores)

        

    # print(train_losses_list, val_losses_list)
    if ensemble == 1:
        return train_losses_list[0], val_losses_list[0], train_accs_list[0], val_accs_list[0], auc_scores_list[0]

    else:
        train_ave_loss = np.average( np.array(train_losses_list), axis = 0) 
        val_ave_loss = np.average( np.array(val_losses_list) , axis = 0 )
        train_ave_acc = np.average( np.array(train_accs_list) , axis = 0)
        val_ave_acc = np.average( np.array(val_accs_list) , axis = 0)

        return   train_ave_loss, val_ave_loss, train_ave_acc, val_ave_acc, auc_scores_list

def validate(model, dataloader, criterion, k):
    model.eval()
    num_correct = 0
    total = 0
    running_loss = 0

    true_labels = np.array([])
    probs = np.array([])

    preds = []
    for batch_idx, (X, Y) in enumerate(dataloader):
        
        output = model.forward(X)
        pred = torch.argmax(output, dim = 1)
        # preds.append(pred)
        num_correct += (pred==Y).sum().item()
        total += Y.size(0)
        loss = criterion(output, Y)
        running_loss += loss.item() * Y.size(0)
        # print("preds", preds)
        probs = np.append(probs, output[:,-1].cpu().detach().numpy())
        true_labels = np.append(true_labels, Y.cpu().detach().numpy())

    auc_scores = roc_auc_score(true_labels , probs) 

    return num_correct/total, running_loss/total, auc_scores


def validate_conf_matrix(model, dataloader, criterion, k):
    model.eval()
    num_correct = 0
    total = 0
    running_loss = 0

    true_labels = np.array([])
    probs = np.array([])

    preds = np.array([])
    for batch_idx, (X, Y) in enumerate(dataloader):
        
        output = model.forward(X)
        pred = torch.argmax(output, dim = 1)
        preds = np.append(preds, pred.cpu().detach().numpy())
        num_correct += (pred==Y).sum().item()
        total += Y.size(0)
        loss = criterion(output, Y)
        running_loss += loss.item() * Y.size(0)
        # print("preds", preds)
        probs = np.append(probs, output[:,-1].cpu().detach().numpy())
        true_labels = np.append(true_labels, Y.cpu().detach().numpy())

    auc_scores = roc_auc_score(true_labels , probs) 

    return num_correct/total, running_loss/total, auc_scores, true_labels, preds


# Model

In [75]:
class SGC(nn.Module):
    """
    A Simple PyTorch Implementation of Logistic Regression.
    Assuming the features have been preprocessed with k-step graph propagation.
    """
    def __init__(self, nfeat, nclass):
        super(SGC, self).__init__()

        self.W = nn.Linear(nfeat, nclass)
        self.sm = nn.Softmax(dim=1)

    def forward(self, x):
        return self.W(x)


def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.weight)
        # m.bias.data.fill_(0.01)

In [76]:
##############
ensemble = 20
##############

# n_feature = 133 + 200
n_feature = 133
n_class = 2
k = 6
num_epoch = 30
lr = 0.001
wd = 5e-6

SGC_base_list = []
optimizer_list = []
scheduler_list = []

for i in range(ensemble):
    SGC_base = SGC(n_feature, n_class)
    SGC_base.apply(init_weights)
    SGC_base_list.append(SGC_base)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(SGC_base.parameters(), lr = lr, weight_decay = wd)
    optimizer_list.append(optimizer)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience = 0)
    scheduler_list.append(scheduler)

In [77]:
# Split data and labels into postive and negative
def split_pos_neg(X_set, Y):
    pos_X_set = []
    neg_X_set = []
    for i in range(len(X_set)):
        if Y[i] == 1:
            X_set_Y = np.append(X_set[i], [1])
            # print(X_set_Y.shape)
            pos_X_set.append(X_set_Y)
        else:
            X_set_Y = np.append(X_set[i], [0])
            neg_X_set.append(X_set_Y)
    return np.array(pos_X_set), np.array(neg_X_set)

train_f = np.load("data/train_feature.npy", allow_pickle = True)
train_adj_mx = np.load("data/train_adjacent_matrix.npy", allow_pickle = True)
train_mol_f = np.load("data/train_mol_features.npy", allow_pickle = True)
train_label = np.load("data/train_label.npy", allow_pickle = True)

# print("train_f", train_f.shape)
# print("train_adj_mx", train_adj_mx.shape)
# print("train_mol", train_mol_f.shape)

# concatenate atom features and adjancency matrix
train_dataloader_list = []
val_dataloader_list = []


concat_f = []
for count in range(train_f.shape[0]):   #2335
    one_mol_f = []
    one_mol_f.extend( [train_f[count], train_adj_mx[count], train_mol_f[count] ] )
    concat_f.append(one_mol_f)

concat_f = np.asarray(concat_f)
train_X_set, val_X_set, train_Y, val_Y = model_selection.train_test_split(concat_f, train_label, test_size = 0.2, random_state = 42)

pos_X_set, neg_X_set = split_pos_neg(train_X_set, train_Y)  # Take all 

print(pos_X_set.shape, neg_X_set.shape)

for i in range(ensemble):  
    # For each ensemble, pick all positive samples and same number of negative samples. (balanced 1:1)
    mol_number = neg_X_set.shape[0]
    random_indices = np.random.choice(mol_number, size=len(pos_X_set)*2 , replace=False)
    part_neg_X_set = neg_X_set[random_indices]

    train_X_set =  np.concatenate((pos_X_set, part_neg_X_set))
    # print(train_X_set[:, 3])

    np.random.shuffle(train_X_set)
    # print(train_X_set[:, 3])

    train_X = train_X_set[:,0]
    train_A = train_X_set[:,1]
    train_mol_X = train_X_set[:,2]
    train_Y = train_X_set[:,3]   # This partial train_Y overwrites the overall train_Y before.

    val_X = val_X_set[:,0]
    val_A = val_X_set[:,1]
    val_mol_f = val_X_set[:,2]

    degree = 8

    train_dataset = MyDataset_mol(train_X, train_A, train_Y, degree, train_mol_X)
    val_dataset = MyDataset_mol(val_X, val_A, val_Y, degree, val_mol_f)

    train_dataloader_args = dict(shuffle = True, batch_size = 8, drop_last = True)
    val_dataloader_args = dict(shuffle = False, batch_size = 8, drop_last = True)
    train_dataloader = DataLoader(train_dataset, **train_dataloader_args)
    val_dataloader = DataLoader(val_dataset, **val_dataloader_args)

    train_dataloader_list.append(train_dataloader)
    val_dataloader_list.append(val_dataloader)


(100, 4) (1768, 4)


# Run Model

In [ ]:
train_losses, val_losses, train_accs, val_accs, auc_scores = train(SGC_base_list, train_dataloader_list, val_dataloader_list, 
                                                       optimizer_list, scheduler_list, criterion, num_epoch, k, ensemble)

training ensemble model 1
training ensemble model 2
training ensemble model 3
training ensemble model 4
training ensemble model 5
training ensemble model 6
training ensemble model 7
training ensemble model 8
training ensemble model 9
training ensemble model 10
training ensemble model 11
training ensemble model 12
training ensemble model 13
training ensemble model 14
training ensemble model 15
training ensemble model 16
training ensemble model 17
training ensemble model 18


In [ ]:
model = SGC_base_list[0]
# for param in model.parameters():
#   print(param.data.shape)
#   print(param.data)

In [ ]:
plt.plot(train_losses, label = "train_loss")
plt.plot(val_losses, label = "val_loss")
# plt.plot(train_accs, label = "train_acc")
# plt.plot(val_accs, label = "val_acc")
plt.legend()
plt.xlabel("training epoch")

# plt.savefig("/content/drive/MyDrive/Deep Learning Project/output_fig/ensemble_mol_loss.jpg")
# plt.savefig("/content/drive/MyDrive/Deep Learning Project/output_fig/ensemble_mol_acc.jpg")

# print("train_accs", train_accs)
# print("val_accs", val_accs)
# plt.show()

In [ ]:
test_X = np.load("data/test_feature.npy", allow_pickle = True)
test_A = np.load("data/test_adjacent_matrix.npy", allow_pickle = True)
test_mol_f = np.load("data/test_mol_features.npy", allow_pickle = True)
test_Y = np.load("data/test_label.npy", allow_pickle = True)


test_dataset = MyDataset_mol(test_X, test_A, test_Y, k, test_mol_f)
test_dataloader = DataLoader(test_dataset)

In [ ]:
import seaborn as sns

test_acc_list = []
auc_list = []
# print(SGC_base_list)


for i in range(ensemble):
    SGC_base = SGC_base_list[i]
    test_acc, _, auc_score, true_labels, preds  = validate_conf_matrix(SGC_base, test_dataloader, criterion, k)
    test_acc_list.append(test_acc)
    auc_list.append(auc_score)

print("label",list(test_Y))

cm = confusion_matrix(true_labels, preds)
print(cm)
# plt.imshow(cm, cmap='binary')

matrix_df = pd.DataFrame(cm, index=['True_Inactive', 'True_Active'], columns=['Pred_Inactive', 'Pred_Active'])
ax=sns.heatmap(matrix_df, cmap='YlGnBu', annot=True, fmt='d', annot_kws={'size': 16}).set_title('SGC ensemble Confusion Matrix')
plt.savefig('/content/drive/MyDrive/Deep Learning Project/output_fig/{}_matrix.png'.format("SGC_ensmeble"))
plt.show()

In [ ]:
print(test_acc_list)
print(sum(test_acc_list)/ensemble)
print(auc_list)